In [1]:
import pandas as pd
import numpy as np

# Gerar Inserts Projeto BD1

## Carregar dados no dataframe disciplina e pré-processar colunas

In [2]:
disciplina = pd.read_excel("dados/disciplinasDCET.xlsx")

disciplina['COD'] = disciplina['COD'].replace(" ", "")
disciplina.columns = disciplina.columns.str.lower()
disciplina = disciplina.rename(columns={"b ou l": "formacao", "área": "area"})
disciplina['id_departamento'] = 5
disciplina.loc[disciplina['area'].isna(), 'area'] = 'oceanografia'

# Replace some Nan values
disciplina.loc[407, 'ch'] = 60
disciplina.loc[559, 'ch'] = 90
disciplina.loc[730, 'ch'] = 60

columns = ['area', 'curso', 'disciplina', 'formacao']

for column in columns:
    disciplina[column] = disciplina[column].str.lower().str.strip()

mask = disciplina['formacao'] == 'b e l'
disciplina.loc[mask, 'formacao'] = 'b+l'
disciplina.loc[disciplina['area'] == 'dcet', 'area'] = 'oceanografia'
mask = disciplina['formacao'] == 'b+l'
tmp = disciplina.loc[mask].copy()
tmp.formacao = 'licenciatura'
disciplina.loc[mask, 'formacao'] = 'bacharelado'
disciplina = pd.concat([disciplina, tmp], axis=0).reset_index(drop=True)
disciplina.formacao = disciplina.formacao.replace(to_replace=['b', 'l'], value=['bacharelado', 'licenciatura'])

disciplina

,cod,disciplina,ch,formacao,curso,area,id_departamento
0,CET154,química geral e orgânica,90.0,bacharelado,biomedicina,química,5
1,CET558,calculo aplicado a biomedicina,60.0,bacharelado,biomedicina,matemática,5
2,CET559,física aplicada a biomedicina,60.0,bacharelado,biomedicina,física,5
3,CIB520,bioestatistica,60.0,bacharelado,biomedicina,estatística,5
4,CIB517,bioinformática,60.0,bacharelado,biomedicina,computação,5
...,...,...,...,...,...,...,...
802,CET020,bioestatística,60.0,licenciatura,ciências biológicas,estatística,5
803,CET561,matemática para biociências,60.0,licenciatura,ciências biológicas,matemática,5
804,CET647,fisíca para biociências,45.0,licenciatura,ciências biológicas,física,5
805,CET650,quimíca para biociências,60.0,licenciatura,ciências biológicas,química,5


## Carregar dataframes

In [3]:
departamento = pd.read_csv("dados/departamentos.csv")
departamento.sigla = departamento.sigla.str.lower()
departamento.nome = departamento.nome.str.lower()

docente = pd.read_csv("dados/docentes.csv")
docente.nome = docente.nome.str.lower()

pro_reitoria = pd.read_csv("dados/pro_reitorias.csv")
pro_reitoria.sigla = pro_reitoria.sigla.str.lower()
pro_reitoria.nome = pro_reitoria.nome.str.lower()

semestre = pd.read_csv("dados/semestres.csv")
semestre.semestre = semestre.semestre.astype("str")

colegiado = pd.read_csv("dados/colegiados.csv")
colegiado.colegiado = colegiado.colegiado.str.lower()

area = pd.read_csv("dados/areas.csv")
cargo = pd.read_csv("dados/cargo.csv")
titulo = pd.read_csv("dados/titulo.csv")
curso = pd.read_csv("dados/cursos.csv")
formacao = pd.read_csv("dados/formacoes.csv")
curso_formacao = pd.read_csv("dados/curso_formacao.csv")
disciplina_formacao = pd.read_csv("dados/disciplina_formacao.csv")
turma = pd.read_csv("dados/turma.csv")
aula = pd.read_csv("dados/aulas.csv")
doc_indicado = pd.read_csv("dados/docentes_indicados.csv")
tipo_aula = pd.read_csv("dados/tipo_aula.csv")
docentes = pd.read_csv("dados/docentes.csv")

## Criar colunas PK e FK

In [4]:
def create_fk(df_fk, column_fk):
    fk_values = {df_fk.loc[i, column_fk]: i + 1 for i in range(df_fk.shape[0])}
    return fk_values
        
disciplina.curso = disciplina.curso.map(create_fk(curso, 'curso'))
disciplina.area = disciplina.area.map(create_fk(area, 'area'))
disciplina.formacao = disciplina.formacao.map(create_fk(formacao, 'formacao'))

tmp = pd.DataFrame({"cod":disciplina.cod.unique()})
index = pd.DataFrame({"id": disciplina.cod.map(create_fk(tmp, 'cod'))})
disciplina = pd.concat([index, disciplina], axis=1)

new_names_columns = {"area": "id_area", "curso": "id_curso", "formacao": "id_formacao"}
disciplina = disciplina.rename(columns=new_names_columns)

## Finalizar pré-processamento disciplina

In [5]:
disciplina = disciplina.drop(["id_formacao","id_curso"], axis=1).drop_duplicates(subset="cod").reset_index(drop=True)
id_area = disciplina.id_area.copy(deep=True)
disciplina = disciplina.drop(columns=['id_area', 'id_departamento'])
disciplina = pd.concat([disciplina, id_area], axis=1)
disciplina.to_csv("dados/disciplinas.csv", index=False)
disciplina

,id,cod,disciplina,ch,id_area
0,1,CET154,química geral e orgânica,90.0,35
1,2,CET558,calculo aplicado a biomedicina,60.0,34
2,3,CET559,física aplicada a biomedicina,60.0,33
3,4,CIB520,bioestatistica,60.0,32
4,5,CIB517,bioinformática,60.0,26
...,...,...,...,...,...
580,581,CET497,métodos quantitativos aplicados à administraçã...,60.0,34
581,582,CET117,introdução à estatística,60.0,32
582,583,CET498,estatística aplicada,60.0,32
583,584,CET1234,estatistica aplicada a saúde,60.0,32


## Gerar e Salvar inserts

In [6]:
def create_insert(df, nome_tabela) -> list:
    inserts = []
    
    for i in range(df.shape[0]):
        values = []
        atributos = []
        for column in df.columns:
            atributos.append(f"{column}")
              
            if isinstance(df.loc[i, column], str):
                if column in ['dt_inicio', 'dt_fim']: 
                    if df.loc[i, column] == 'NULL':
                        values.append(f"NULL")
                    else:
                        values.append(f"to_date('{df.loc[i, column]}', 'DD/MM/YYYY')")
                else:
                    values.append(f"'{df.loc[i, column]}'")
            else:
                values.append(f"{df.loc[i, column]}")
                
        values = ", ".join(values)
        atributos = ", ".join(atributos)
        insert = f"insert into {nome_tabela} ({atributos}) values ({values});\n"
        inserts.append(insert)
    return inserts

inserts = create_insert(departamento, "departamento")
inserts += create_insert(pro_reitoria, "pro_reitoria")
inserts += create_insert(semestre, "semestre")
inserts += create_insert(formacao, "formacao")
inserts += create_insert(colegiado, "colegiado")
inserts += create_insert(curso, "curso")
inserts += create_insert(area, "area")
inserts += create_insert(disciplina, "disciplina")
inserts += create_insert(docente, "docente")
inserts += create_insert(disciplina_formacao, "disciplina_formacao")
inserts += create_insert(curso_formacao, "curso_formacao")
inserts += create_insert(turma, "turma")
inserts += create_insert(aula, "aula")
inserts += create_insert(doc_indicado, "doc_indicado")
inserts += create_insert(tipo_aula, "tipo_aula")

with open("sql/inserts.sql", "w") as file:
    for insert in inserts:
        file.write(insert)

' def create_insert(df, nome_tabela) -> list:\n    inserts = []\n    \n    for i in range(df.shape[0]):\n        values = []\n        atributos = []\n        for column in df.columns:\n            atributos.append(f"{column}")\n                \n            if isinstance(df.loc[i, column], str):\n                if df.loc[i, column] == \'NULL:\n                    values.append(f"NULL")\n                else:\n                    values.append(f"\'{df.loc[i, column]}\'")\n            else:\n                values.append(f"{df.loc[i, column]}")\n                \n        values = ", ".join(values)\n        atributos = ", ".join(atributos)\n        insert = f"insert into {nome_tabela} ({atributos}) values ({values});\n"\n        inserts.append(insert)\n    return inserts\n\ninserts = create_insert(departamento, "departamento")\ninserts += create_insert(pro_reitoria, "pro_reitoria")\ninserts += create_insert(semestre, "semestre")\ninserts += create_insert(formacao, "formacao")\ninserts +=